# PlatonovGen - Character-Level LSTM

Training a neural recurrent network for generating Andrei Platonov-style texts.

In [5]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

## Data Loading

Let's load the dataset Platonov8, consisting of the 8-volume collection of the writer's works

In [6]:
!wget https://raw.githubusercontent.com/moxeeem/PlatonovGen/main/Platonov8.txt

--2024-02-03 19:45:31--  https://raw.githubusercontent.com/moxeeem/PlatonovGen/main/Platonov8.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12291197 (12M) [text/plain]
Saving to: ‘Platonov8.txt’

Platonov8.txt       100%[===================>]  11.72M  66.3MB/s    in 0.2s    

2024-02-03 19:45:35 (66.3 MB/s) - ‘Platonov8.txt’ saved [12291197/12291197]



In [7]:
with open("Platonov8.txt", "r") as f:
    text = f.read()

Let's look at first 100 characters:

In [8]:
text[:100]

'Издревле и повсесюдно все старики спят. Спят так, что пузыри от уст отскакивают и одиноко мокнет поз'

### Tokenization

Let's create a couple dictionaries to convert the characters to and from integers.

In [9]:
# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))

int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

Let's see how the characters are encoded with integers

In [10]:
encoded[:100]

array([110,  80, 134,  99,  56,  39, 128,  56,  47,  72,  47, 140,  95,
        39,  44,  56,  44, 108, 134,  85,  95,  47,  39,  44,  56,  47,
        44, 151,  96,  99,  72,  53,  72,  47,  44, 140,  14, 151, 125,
        47, 124, 140,  14, 151,  47, 151,  96,  53,  15,  47,   0, 151,
        95,  47, 140,   5,  80,  73,  99,  72,  47,  95, 151,  47,   5,
        44, 151,  47,  95, 151,  44,  53,  96,  53,  72,  39,  96, 108,
       151,  47,  72,  47,  95, 134,  72,  85,  95,  53,  95,  47, 103,
        95,  53,  85,  56, 151,  47, 140,  95,  80])

## Preprocessing

LSTM expects **one-hot encoded** input, which means that each character is converted to an integer and then converted to a column vector where only its corresponding integer index will have a value of 1 and the rest of the vector will be filled with zeros. Let's create a function for this.

In [11]:
def one_hot_encode(arr, n_labels):

    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.0

    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot

In [12]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making Training Mini-Batches

In [13]:
def get_batches(int_words, batch_size, seq_length):
    """
    Generates batches from encoded sequence.
    :param int_words: tensor of ints, which represents the text; shape: [batch_size, -1]
    :param batch_size: number of sequences per batch
    :param seq_length: number of encoded chars in a sequence
    :return: generator of pairs (x, y); x_shape, y_shape: [batch_size, seq_length - 1]
    """
    # 1. Truncate text, so there are only full batches
    window_size = seq_length + 1
    batch_size_total = batch_size * window_size
    n_batches = len(int_words) // batch_size_total
    int_words = int_words[: n_batches * batch_size_total]

    # 2. Reshape into batch_size rows
    int_words = int_words.reshape((batch_size, -1))

    # 3. Iterate through the text matrix
    for position in range(0, int_words.shape[1], window_size):
        x = int_words[:, position : position + window_size - 1]
        y = int_words[:, position + 1 : position + window_size]
        yield x, y

### Test

Let's create some datasets, and check what happens when we create batches.

In [14]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [15]:
# printing out the first 10 items in a sequence
print("x\n", x[:10, :10])
print("\ny\n", y[:10, :10])

x
 [[110  80 134  99  56  39 128  56  47  72]
 [ 47 151  96  53  47  85  72  47  53  47]
 [ 56 128  95  39  56  53  96  47 144  47]
 [116  56  47  85  56  47   5  44 140  56]
 [ 47  45  95  99  95 134  56  47  85  96]
 [ 72  56  47  80  96  47  39  99  56 103]
 [ 47 129  96  80  47  72 128  72  47 134]
 [ 85  85  95  47  85  96  99  95 134  85]]

y
 [[ 80 134  99  56  39 128  56  47  72  47]
 [151  96  53  47  85  72  47  53  47   0]
 [128  95  39  56  53  96  47 144  47  39]
 [ 56  47  85  56  47   5  44 140  56 128]
 [ 45  95  99  95 134  56  47  85  96  47]
 [ 56  47  80  96  47  39  99  56 103  14]
 [129  96  80  47  72 128  72  47 134  39]
 [ 85  95  47  85  96  99  95 134  85  95]]


---
## Network Setting


In [16]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    print("Training on GPU!")
else:
    print("No GPU available, training on CPU; consider making n_epochs very small.")

Training on GPU!


In [17]:
class CharRNN(nn.Module):
    def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
      #drop_prob=0.5
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## define the LSTM
        self.lstm = nn.LSTM(
            len(self.chars), n_hidden, n_layers, dropout=drop_prob, batch_first=True
        )

        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)

        ## define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))

    def forward(self, x, hidden):
        """Forward pass through the network.
        These inputs are x, and the hidden/cell state `hidden`."""

        ## Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)

        ## pass through a dropout layer
        out = self.dropout(r_output)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)

        ## put x through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden

    def init_hidden(self, batch_size):
        """Initializes hidden state"""
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if train_on_gpu:
            hidden = (
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
            )
        else:
            hidden = (
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
            )

        return hidden

## Network Training

In [18]:
def train(
    net,
    data,
    epochs=10,
    batch_size=10,
    seq_length=50,
    lr=0.001,
    clip=5,
    val_frac=0.1,
    print_every=10,
):
    """Training a network

    Arguments
    ---------

    net: CharRNN network
    data: text data to train the network
    epochs: Number of epochs to train
    batch_size: Number of mini-sequences per mini-batch, aka batch size
    seq_length: Number of character steps per mini-batch
    lr: learning rate
    clip: gradient clipping
    val_frac: Fraction of data to hold out for validation
    print_every: Number of steps for printing training and validation loss

    """
    net.train()

    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data) * (1 - val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if train_on_gpu:
        net.cuda()

    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)

        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1

            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if train_on_gpu:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size * seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if train_on_gpu:
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(
                        output, targets.view(batch_size * seq_length).long()
                    )

                    val_losses.append(val_loss.item())

                net.train()  # reset to train mode after iterationg through validation data

                print(
                    "Epoch: {}/{}...".format(e + 1, epochs),
                    "Step: {}...".format(counter),
                    "Loss: {:.4f}...".format(loss.item()),
                    "Val Loss: {:.4f}".format(np.mean(val_losses)),
                )


        # Save the model every 10 epochs
        if (e + 1) % 10 == 0:
            checkpoint = {
                "n_hidden": net.n_hidden,
                "n_layers": net.n_layers,
                "state_dict": net.state_dict(),
                "tokens": net.chars,
            }
            with open(f"checkpoint_{e + 1}_epoch.net", "wb") as f:
                torch.save(checkpoint, f)

## Network Definition


In [19]:
# define and print the net
n_hidden = 1024
n_layers = 3

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(152, 1024, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=1024, out_features=152, bias=True)
)


### Hyperparameters Setting and Training

In [ ]:
batch_size = 128
seq_length = 100
n_epochs = 20

# train the model
train(
    net,
    encoded,
    epochs=n_epochs,
    batch_size=batch_size,
    seq_length=seq_length,
    lr=0.001,
    print_every=10,
)


Epoch: 1/20... Step: 10... Loss: 3.4097... Val Loss: 3.3195
Epoch: 1/20... Step: 20... Loss: 3.3985... Val Loss: 3.3023
Epoch: 1/20... Step: 30... Loss: 3.3815... Val Loss: 3.2940
Epoch: 1/20... Step: 40... Loss: 3.3482... Val Loss: 3.2894
Epoch: 1/20... Step: 50... Loss: 3.3326... Val Loss: 3.2943
Epoch: 1/20... Step: 60... Loss: 3.3262... Val Loss: 3.2874
Epoch: 1/20... Step: 70... Loss: 3.3388... Val Loss: 3.2895
Epoch: 1/20... Step: 80... Loss: 3.3348... Val Loss: 3.2894
Epoch: 1/20... Step: 90... Loss: 3.3212... Val Loss: 3.2895
Epoch: 1/20... Step: 100... Loss: 3.3015... Val Loss: 3.2841
Epoch: 1/20... Step: 110... Loss: 3.3196... Val Loss: 3.2876
Epoch: 1/20... Step: 120... Loss: 3.3206... Val Loss: 3.2867
Epoch: 1/20... Step: 130... Loss: 3.3159... Val Loss: 3.2894
Epoch: 1/20... Step: 140... Loss: 3.3261... Val Loss: 3.2871
Epoch: 1/20... Step: 150... Loss: 3.3041... Val Loss: 3.2839
Epoch: 1/20... Step: 160... Loss: 3.2988... Val Loss: 3.2855
Epoch: 1/20... Step: 170... Loss:

## Checkpoint


In [ ]:
model_name = "rnn_x_epoch.net"

checkpoint = {
    "n_hidden": net.n_hidden,
    "n_layers": net.n_layers,
    "state_dict": net.state_dict(),
    "tokens": net.chars,
}

with open(model_name, "wb") as f:
    torch.save(checkpoint, f)

---
## Making Prediction

In [21]:
def predict(net, char, h=None, top_k=None):
    """Given a character, predict the next character.
    Returns the predicted character and the hidden state.
    """

    # tensor inputs
    x = np.array([[net.char2int[char]]])
    x = one_hot_encode(x, len(net.chars))
    inputs = torch.from_numpy(x)

    if train_on_gpu:
        inputs = inputs.cuda()

    # detach hidden state from history
    h = tuple([each.data for each in h])
    # get the output of the model
    out, h = net(inputs, h)

    # get the character probabilities
    p = F.softmax(out, dim=1).data
    if train_on_gpu:
        p = p.cpu()  # move to cpu

    # get top characters
    if top_k is None:
        top_ch = np.arange(len(net.chars))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p / p.sum())

    # return the encoded value of the predicted char and the hidden state
    return net.int2char[char], h

### Priming and Text Generating

In [22]:
def sample(net, size, prime="Начало", top_k=None):

    if train_on_gpu:
        net.cuda()
    else:
        net.cpu()

    net.eval()  # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)

    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return "".join(chars)

In [ ]:
print(sample(net, 1000, prime="Добрый день", top_k=2))

Добрый день по полу по своему получению своей силы. В тело старые страны пространства на столе поставили с подоконника в подводной поляне. В том жилище он поднял его, прочел и подошел к светлому стеклу, поднимал его свою силу и поднялся на пол и скрывался на пол и пошел в свое поле. Но он стал принимать в ней не столько свои страны и прочие привыкли к своим солнцем.

— Ты стал, товарищ Чепурный? — спросил он. — Ты прочь с нами, а ты стал старым и после своего солдата.

— Ну как ты сколько ты, ты подождешь? — сказал Сербинов и спросил его про себя:

— А я с ними полюбил — только в светлом поле начнется. А вы не помните, какое тело продолжается из своих пространств и нашего состава нашей привязанности к ней и получить, — скажи мне по сторону положения, а не в темноте станции.

— Нет, я не приду на свете!

— Ну как же, товарищ капитан, — сказал он. — Ты скоро умру не старуху, а ты не помнишь меня, ты не привыкла бы привыкну к сердце и придется страшно. Но ты не помнишь меня, ты скажи мне 

From the generated text we can draw the following conclusions.

First, when using CharRNN it is rather hard to achieve meaningfulness of the text.
In most cases we failed to achieve meaningfulness, but sometimes words are formed correctly,
for example: "на столе поставили", "в том жилище он поднял его, прочел и подошел к светлому стеклу".

In addition, we managed to achieve stylistically good generation. Platonov's texts have always possessed
"arbitrariness in the combination of words," as many critics have noted. This is what we see: "тело продолжается из своих
пространств", "состава нашей привязанности к ней".

In addition, we see that the neural network has a good grasp of the punctuation of the Russian language,
which is evident in the grammatically correct dialogs.

With morphemics things are worse, the text contains many words that are constructed incorrectly,
such as "ты скоро умрУ".

## Loading a checkpoint

In [24]:
# Here we have loaded in a model
with open("rnn_x_epoch.net", "rb") as f:
    checkpoint = torch.load(f)

loaded = CharRNN(
    checkpoint["tokens"],
    n_hidden=checkpoint["n_hidden"],
    n_layers=checkpoint["n_layers"],
)
loaded.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [29]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="Погода была"))

Погода была в самом деле и на поле, не превращающиеся к ней.

— А ты в культуре составил, а ты сейчас? А ты страшно теперь, а не под конец…

В сапог он никогда не успел подняться; он посмотрел на мальчика, потому что неизвестно было в пустоте небольшой девичьих ламп, потому что водой в степе стало скрытое плодное стекло. По своему случаю полезно искали остаток. Но нет того, что страдало и не может стать правыми словами. В стороне немцев, который не знал в своей пищевой судьбе, а по всей дороге было скончаться не при старых и последнего чужая. На всю слободу на две трудные вещи паровозных стоит стояли трамвае, по небольшому тому и стану не будут справлять искусства. Но несколько старик стал думать о ней и приказал себя вскоре обратить на свою страсть и на пустой петухов, искать ее и скупо и неподвижно. Прохожий красноармеец пришел к старости, все рассеянно склонился к своему воздуху.

— Ну дочь, — ответил Соплян и стал несли старое действие.

Перин поступил подпользоваться на степной до